In [1]:
!curl -L -o food-com-recipes-and-user-interactions.zip\
  https://www.kaggle.com/api/v1/datasets/download/shuyangli94/food-com-recipes-and-user-interactions


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  267M  100  267M    0     0  2559k      0  0:01:46  0:01:46 --:--:-- 1717k     0  0:02:00  0:00:06  0:01:54 2655k:01:42  0:00:09  0:01:33 3339kk      0  0:01:29  0:00:29  0:01:00 3201k0     0  2960k      0  0:01:32  0:00:33  0:00:59 2455kM    0     0  2867k      0  0:01:35  0:00:38  0:00:57 2237k 0  0:01:45  0:01:32  0:00:13 2665k


In [2]:
import zipfile
import pandas as pd
with zipfile.ZipFile('food-com-recipes-and-user-interactions.zip', 'r') as zf:
    # Available files
    print(zf.namelist())
    df_recipes = pd.read_csv(zf.open('RAW_recipes.csv'))

['PP_recipes.csv', 'PP_users.csv', 'RAW_interactions.csv', 'RAW_recipes.csv', 'ingr_map.pkl', 'interactions_test.csv', 'interactions_train.csv', 'interactions_validation.csv']


In [3]:
df_recipes

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8
...,...,...,...,...,...,...,...,...,...,...,...,...
231632,zydeco soup,486161,60,227978,2012-08-29,"['ham', '60-minutes-or-less', 'time-to-make', ...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22
231633,zydeco spice mix,493372,5,1500678,2013-01-09,"['15-minutes-or-less', 'time-to-make', 'course...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13
231634,zydeco ya ya deviled eggs,308080,40,37779,2008-06-07,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8
231635,cookies by design cookies on a stick,298512,29,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10


In [8]:
import pandas as pd
import re
from src.keywords import KEYWORDS

SEED = 30
N = 500

# Filter recipes containing at least one keyword
df_filtered = df_recipes[
    df_recipes['tags'].str.contains('|'.join(KEYWORDS), case=False, na=False)
]

# Extract the first matching keyword for grouping
df_filtered['keyword_match'] = df_filtered['tags'].str.extract(
    '(' + '|'.join(KEYWORDS) + ')', 
    flags=re.IGNORECASE
)

# Calculate samples per group with minimum handling
samples_per_keyword = N // len(KEYWORDS)

# Stratified sampling with handling for small groups
df_sampled = (
    df_filtered
    .groupby('keyword_match', group_keys=False)
    .apply(lambda x: x.sample(
        n=min(len(x), samples_per_keyword), 
        random_state=SEED
    ))
    .reset_index(drop=True)
)

# Drop the temporary column
df_sampled = df_sampled.drop(columns=['keyword_match'])
df_sampled

# Save the sampled dataset
df_sampled.to_csv('data/small_recipes.csv', index=False)

/var/folders/tz/0j3yjkdj7nvfwmbmk05fxk5w0000gn/T/ipykernel_51982/3129485718.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['keyword_match'] = df_filtered['tags'].str.extract(
/var/folders/tz/0j3yjkdj7nvfwmbmk05fxk5w0000gn/T/ipykernel_51982/3129485718.py:26: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(


In [9]:
from src.keywords import KEYWORDS
import pandas as pd

# Count occurrences of each keyword across all rows
keyword_counts = {}
for keyword in KEYWORDS:
    keyword_counts[keyword] = df_sampled['tags'].str.contains(
        keyword, case=False, na=False
    ).sum()

keyword_freq = pd.Series(keyword_counts).sort_values(ascending=False)
print(keyword_freq)


vegetarian            148
low-carb              121
inexpensive           120
healthy                66
vegan                  65
gluten-free            56
high-protein           55
kosher                 47
diabetic               45
15-minutes-or-less     45
lactose-free            0
dtype: int64
